In [45]:
from pandas import DataFrame
import pandas
import requests
from IPython import ipywidgetswidgets

ImportError: cannot import name 'ipywidgets'

In [8]:
frame = pandas.read_csv("fortune500.csv")
frame

,lon,lat,company,location,industry,state,city
0,-94.2088,36.3729,Walmart,"Bentonville, AR",General Merchandisers,AR,Bentonville
1,-96.9489,32.8140,Exxon Mobil,"Irving, TX",Petroleum Refining,TX,Irving
2,-121.9780,37.7799,Chevron,"San Ramon, CA",Petroleum Refining,CA,San Ramon
3,-95.9980,41.2524,Berkshire Hathaway,"Omaha, NE",Insurance: Property and Casualty (Stock),NE,Omaha
4,-122.0320,37.3230,Apple,"Cupertino, CA","Computers, Office Equipment",CA,Cupertino
5,-83.0458,42.3314,General Motors,"Detroit, MI",Motor Vehicles and Parts,MI,Detroit
6,-95.3698,29.7604,Phillips 66,"Houston, TX",Petroleum Refining,TX,Houston
7,-73.2613,41.1408,General Electric,"Fairfield, CT",Diversified Financials,CT,Fairfield
8,-83.1763,42.3223,Ford Motor,"Dearborn, MI",Motor Vehicles and Parts,MI,Dearborn
9,-71.5148,42.0029,CVS Health,"Woonsocket, RI",Food and Drug Stores,RI,Woonsocket


In [19]:
industries = set(list(frame.ix[:, 'industry']))
industries

{'Advertising, marketing',
 'Aerospace and Defense',
 'Airlines',
 'Apparel',
 'Automotive Retailing, Services',
 'Beverages',
 'Building Materials, Glass',
 'Chemicals',
 'Commercial Banks',
 'Computer Peripherals',
 'Computer Software',
 'Computers, Office Equipment',
 'Construction and Farm Machinery',
 'Diversified Financials',
 'Diversified Outsourcing Services',
 'Education',
 'Electronics, Electrical Equip.',
 'Energy',
 'Engineering, Construction',
 'Entertainment',
 'Financial Data Services',
 'Food Consumer Products',
 'Food Production',
 'Food Services',
 'Food and Drug Stores',
 'Forest and Paper Products',
 'General Merchandisers',
 'Health Care: Insurance and Managed Care',
 'Health Care: Medical Facilities',
 'Health Care: Pharmacy and Other Services',
 'Home Equipment, Furnishings',
 'Homebuilders',
 'Hotels, Casinos, Resorts',
 'Household and Personal Products',
 'Industrial Machinery',
 'Information Technology Services',
 'Insurance: Life, Health (Mutual)',
 'Insuranc

In [38]:
dat = frame.ix[:, ['company', 'industry']].to_json(orient='values')
dat

'[["Walmart","General Merchandisers"],["Exxon Mobil","Petroleum Refining"],["Chevron","Petroleum Refining"],["Berkshire Hathaway","Insurance: Property and Casualty (Stock)"],["Apple","Computers, Office Equipment"],["General Motors","Motor Vehicles and Parts"],["Phillips 66","Petroleum Refining"],["General Electric","Diversified Financials"],["Ford Motor","Motor Vehicles and Parts"],["CVS Health","Food and Drug Stores"],["McKesson","Wholesalers: Health Care"],["AT&T","Telecommunications"],["Valero Energy","Petroleum Refining"],["UnitedHealth Group","Health Care: Insurance and Managed Care"],["Verizon","Telecommunications"],["AmerisourceBergen","Wholesalers: Health Care"],["Fannie Mae","Diversified Financials"],["Costco","Specialty Retailers: Other"],["HP","Computers, Office Equipment"],["Kroger","Food and Drug Stores"],["JP Morgan Chase","Commercial Banks"],["Express Scripts Holding","Health Care: Pharmacy and Other Services"],["Bank of America Corp.","Commercial Banks"],["IBM","Informa

In [20]:
%%javascript

require.config({
    paths: {
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3'
    }
});

<IPython.core.display.Javascript object>

In [40]:
industries

{'Advertising, marketing',
 'Aerospace and Defense',
 'Airlines',
 'Apparel',
 'Automotive Retailing, Services',
 'Beverages',
 'Building Materials, Glass',
 'Chemicals',
 'Commercial Banks',
 'Computer Peripherals',
 'Computer Software',
 'Computers, Office Equipment',
 'Construction and Farm Machinery',
 'Diversified Financials',
 'Diversified Outsourcing Services',
 'Education',
 'Electronics, Electrical Equip.',
 'Energy',
 'Engineering, Construction',
 'Entertainment',
 'Financial Data Services',
 'Food Consumer Products',
 'Food Production',
 'Food Services',
 'Food and Drug Stores',
 'Forest and Paper Products',
 'General Merchandisers',
 'Health Care: Insurance and Managed Care',
 'Health Care: Medical Facilities',
 'Health Care: Pharmacy and Other Services',
 'Home Equipment, Furnishings',
 'Homebuilders',
 'Hotels, Casinos, Resorts',
 'Household and Personal Products',
 'Industrial Machinery',
 'Information Technology Services',
 'Insurance: Life, Health (Mutual)',
 'Insuranc

In [42]:
%%javascript

<IPython.core.display.Javascript object>

In [39]:
%%javascript
require(['d3'], function(d3){
    /*
        Jupyter notebooks save their state between runtimes
        so the chart needs to be explicitly destroyed and
        recreated every time the code is run.
    */
    $("#chart").remove();
    element.append("<div id='chart'></div>");

    
    var dataset = [ 5, 10, 15, 20, 25 ];
    
    console.log(industries)
});

<IPython.core.display.Javascript object>